In [1]:
## Imports
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize
from ast import literal_eval

In [2]:
DATASET_NAME = 'DBLP-ACM'

In [3]:
class LSH :
  
    # Random limits, const
    __max_rand =  1000
    __min_rand = -1000
    
    # Constructor
    def __init__(self, k, L, embedding_size = 150):
        
        # Number of hash function
        self.k = k
        
        # Number of attempts
        self.L = L
        
        # Embedding length 
        self.embedding_size = embedding_size
        
        # Random matrices
        self.normalized_random_matrices = []
        
        for i in range(self.L):
            random_matrix = np.random.randint(self.__min_rand, self.__max_rand,(self.k, self.embedding_size));
            
            # Append normalized random matrices
            self.normalized_random_matrices.append(normalize(random_matrix, axis=1, norm='l1'))
        
    
    # Locality Sensitive hash function
    def locality_sensitive_hash(self, embedding, matrix_index):
        out = 0
      
        for h in self.normalized_random_matrices[matrix_index]:
            if (np.dot(h, embedding) >= 0):
                out = (out << 1) | 1
            else:
                out = (out << 1) | 0

        return out
      
    # Divide in buckets using L-th matrix
    def divide_in_buckets(self, embeddings, matrix_index):
        out = {}
        for embedding in embeddings:
            hash = self.locality_sensitive_hash(embedding, matrix_index)
            if (hash in out):
                out[hash].append(embedding)
            else:
                out[hash] = [embedding]
            
        return out  


In [4]:
test = LSH(k=5, L=2, embedding_size=10)
print(test.normalized_random_matrices)

print(test.locality_sensitive_hash([1,2,3,4,5,6,7,8,9,10], 1))

[array([[-0.14205246,  0.01161767, -0.11582468,  0.07516282,  0.16000704,
        -0.06037669, -0.15595846,  0.08378807, -0.06724168, -0.12797043],
       [ 0.0684727 , -0.11774744,  0.21267065, -0.05738055,  0.08148464,
         0.02367747,  0.13011945, -0.159343  ,  0.09001706, -0.05908703],
       [-0.1000899 ,  0.06323045, -0.12406353, -0.12076716,  0.07132155,
        -0.14204375,  0.1350015 , -0.08960144, -0.13035661,  0.02352412],
       [ 0.02912621, -0.12962017, -0.09708738, -0.11463124,  0.0183955 ,
         0.15329586, -0.14001022, -0.14018055,  0.00902742, -0.16862545],
       [-0.05601454, -0.05469266,  0.10690681, -0.14177132, -0.15052875,
        -0.03040317,  0.13912756, -0.09666226, -0.09418374,  0.12970919]]), array([[-0.17462334,  0.07242694, -0.16227991,  0.03793792,  0.09747686,
        -0.16010165, -0.02940643, -0.11925939,  0.00471955, -0.14176802],
       [ 0.14876458,  0.11187371, -0.01029513, -0.13332189,  0.16523679,
         0.16489362, -0.00549073,  0.08562

In [5]:
## almost equals
embeddings = [[1.2345,2,3,4,5,6,10.4,8,9,10],[1,2,3,4,5,6,7,8,9,10],[1,2,3,5,5,6,7,8,9,10]]
print(len(test.divide_in_buckets(embeddings, 1)))

## not equals
embeddings = np.random.randint(-10000, 10000,(10000, 10))
print(len(test.divide_in_buckets(embeddings, 1)))


2
32


In [6]:
## TEST BLOCKING PERFORMANCE
#    Basta che per ogni tupla vado a prendere la sua corrispondente, ne calcolo
#     i vari L hash e controllo che almeno uno sia uguale e incremento un
#     contatore. La precisione è contatore/numero di tuple controllate, giusto?
def performance_test(filtered_dataset, k, L, embedding_size):
    
    match_found = 0
    
    lsh = LSH(k, L, embedding_size)
    
    # for each elemt in dataset
    for index, row in filtered_dataset.iterrows():
        x_embedding = np.array(literal_eval(row['left_table']))
        y_embedding = np.array(literal_eval(row['right_table']))
          
        x_hashs = set()
        y_hashs = set()
        for i in range(L):
            x_hashs.add(lsh.locality_sensitive_hash(x_embedding, i))
            y_hashs.add(lsh.locality_sensitive_hash(y_embedding, i))
        
        if (len(set.intersection(x_hashs, y_hashs)) > 0):
            match_found += 1
  
    
    return match_found / len(filtered_dataset.index)

In [7]:
## Open dataset 
df = pd.read_csv('../../lsh-test-data/' + DATASET_NAME + '-embeddings.csv')

## Remove 0 labled
df = df[df.label == 1]

precision_max = 0
k_max = 0
L_max = 0
for k in range(30):
    for L in range(10):
        precision = performance_test(df, k + 1, L + 1, 150)
        print("K: {0}, L: {1}, Precision:{2}".format(k + 1, L + 1, precision))
        if (precision >= precision_max):
            precision_max = precision
            k_max = k + 1
            L_max = L + 1

print("Max precision: {0}, k: {1}, L: {2}".format(precision_max, k_max, L_max))

K: 1, L: 1, Precision:0.9684684684684685
K: 1, L: 2, Precision:1.0
K: 1, L: 3, Precision:1.0
K: 1, L: 4, Precision:1.0
K: 1, L: 5, Precision:1.0
K: 1, L: 6, Precision:1.0
K: 1, L: 7, Precision:1.0
K: 1, L: 8, Precision:1.0
K: 1, L: 9, Precision:1.0
K: 1, L: 10, Precision:1.0
K: 2, L: 1, Precision:0.9166666666666666
K: 2, L: 2, Precision:0.9977477477477478
K: 2, L: 3, Precision:1.0
K: 2, L: 4, Precision:1.0
K: 2, L: 5, Precision:1.0
K: 2, L: 6, Precision:1.0
K: 2, L: 7, Precision:1.0
K: 2, L: 8, Precision:1.0
K: 2, L: 9, Precision:1.0
K: 2, L: 10, Precision:1.0
K: 3, L: 1, Precision:0.8986486486486487
K: 3, L: 2, Precision:0.990990990990991
K: 3, L: 3, Precision:0.9954954954954955
K: 3, L: 4, Precision:1.0
K: 3, L: 5, Precision:1.0
K: 3, L: 6, Precision:1.0
K: 3, L: 7, Precision:1.0
K: 3, L: 8, Precision:1.0
K: 3, L: 9, Precision:1.0
K: 3, L: 10, Precision:1.0
K: 4, L: 1, Precision:0.8896396396396397
K: 4, L: 2, Precision:0.9594594594594594
K: 4, L: 3, Precision:0.990990990990991
K: 4, 

K: 22, L: 2, Precision:0.6486486486486487
K: 22, L: 3, Precision:0.786036036036036
K: 22, L: 4, Precision:0.8400900900900901
K: 22, L: 5, Precision:0.8603603603603603
K: 22, L: 6, Precision:0.8828828828828829
K: 22, L: 7, Precision:0.9234234234234234
K: 22, L: 8, Precision:0.9414414414414415
K: 22, L: 9, Precision:0.9527027027027027
K: 22, L: 10, Precision:0.9436936936936937
K: 23, L: 1, Precision:0.44819819819819817
K: 23, L: 2, Precision:0.6351351351351351
K: 23, L: 3, Precision:0.7972972972972973
K: 23, L: 4, Precision:0.8265765765765766
K: 23, L: 5, Precision:0.8693693693693694
K: 23, L: 6, Precision:0.8873873873873874
K: 23, L: 7, Precision:0.9031531531531531
K: 23, L: 8, Precision:0.9346846846846847
K: 23, L: 9, Precision:0.9459459459459459
K: 23, L: 10, Precision:0.9369369369369369
K: 24, L: 1, Precision:0.4752252252252252
K: 24, L: 2, Precision:0.6306306306306306
K: 24, L: 3, Precision:0.740990990990991
K: 24, L: 4, Precision:0.8175675675675675
K: 24, L: 5, Precision:0.85360360